In [40]:
import numpy as np
import pandas as pd
import emoji

from keras.models import Sequential
from keras.layers import Dense, Input, Dropout, SimpleRNN,LSTM, Activation
from keras.utils import np_utils

import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv('train_emoji.csv',header=None)
test = pd.read_csv('test_emoji.csv',header=None)

In [6]:
train.head()

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN


In [7]:
test.head()

,0,1
0,I want to eat\t,4
1,he did not answer\t,3
2,he got a raise\t,2
3,she got me a present\t,0
4,ha ha ha it was so funny\t,2


In [4]:
emoji_dict = { 0 : ":heart:", 1 : ":baseball:", 2 : ":smile:", 3 : ":disappointed:", 4 : ":fork_and_knife:"}

In [5]:
for ix in emoji_dict.keys():
    print ix,
    print emoji.emojize(emoji_dict[ix], use_aliases=True)

0 ❤
1 ⚾
2 😄
3 😞
4 🍴


In [9]:
X_train = train[0]
Y_train = train[1]

X_test = test[0]
Y_test = test[1]

print X_train.shape, Y_train.shape, X_test.shape, Y_test.shape
print ("-------------------------")
print X_train[0],Y_train[0]

(132,) (132,) (56,) (56,)
-------------------------
never talk to me again 3


In [10]:
for ix in range(X_train.shape[0]):
    X_train[ix] = X_train[ix].split()
    
for ix in range(X_test.shape[0]):
    X_test[ix] = X_test[ix].split()
    
Y_train = np_utils.to_categorical(Y_train)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [11]:
print X_train[0],Y_train[0]

['never', 'talk', 'to', 'me', 'again'] [0. 0. 0. 1. 0.]


In [12]:
np.unique(np.array([len(ix) for ix in X_train]) , return_counts=True)

(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
 array([ 4,  5, 26, 35, 20, 21, 11,  5,  1,  4]))

In [13]:
np.unique(np.array([len(ix) for ix in X_test]) , return_counts=True)

(array([2, 3, 4, 5, 6, 7, 8]), array([ 3, 12, 16, 17,  3,  4,  1]))

In [17]:
embeddings_index = {}

f = open('glove.6B.50d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [20]:
embeddings_index["i"].shape

(50,)

In [24]:
from scipy import spatial
spatial.distance.cosine(embeddings_index["happy"], embeddings_index["sad"])

0.31093674898147583

In [31]:
spatial.distance.cosine(embeddings_index["india"], embeddings_index["delhi"])

0.18572336435317993

In [32]:
spatial.distance.cosine(embeddings_index["france"], embeddings_index["paris"])

0.19746702909469604

In [33]:
embedding_matrix_train = np.zeros((X_train.shape[0], 10, 50))
embedding_matrix_test = np.zeros((X_test.shape[0], 10, 50))

for ix in range(X_train.shape[0]):
    for ij in range(len(X_train[ix])):
        embedding_matrix_train[ix][ij] = embeddings_index[X_train[ix][ij].lower()]
        
for ix in range(X_test.shape[0]):
    for ij in range(len(X_test[ix])):
        embedding_matrix_test[ix][ij] = embeddings_index[X_test[ix][ij].lower()]        

In [34]:
print embedding_matrix_train.shape, embedding_matrix_test.shape

(132, 10, 50) (56, 10, 50)


In [41]:
model = Sequential()
model.add(SimpleRNN(64, input_shape=(10,50), return_sequences=True))
model.add(Dropout(0.5))
model.add(SimpleRNN(64, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, 10, 64)            7360      
_________________________________________________________________
dropout_5 (Dropout)          (None, 10, 64)            0         
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 64)                8256      
_________________________________________________________________
dropout_6 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 325       
_________________________________________________________________
activation_2 (Activation)    (None, 5)                 0         
Total params: 15,941
Trainable params: 15,941
Non-trainable params: 0
_________________________________________________________________


In [43]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [44]:
hist = model.fit(embedding_matrix_train,Y_train,
                epochs = 50, batch_size=32,shuffle=True
                )

Epoch 1/50
132/132 [==============================] - 1s 7ms/step - loss: 1.7987 - acc: 0.2273
Epoch 2/50
132/132 [==============================] - 0s 887us/step - loss: 1.6189 - acc: 0.2576
Epoch 3/50
132/132 [==============================] - 0s 860us/step - loss: 1.5646 - acc: 0.3258
Epoch 4/50
132/132 [==============================] - 0s 773us/step - loss: 1.6140 - acc: 0.2879
Epoch 5/50
132/132 [==============================] - 0s 897us/step - loss: 1.5285 - acc: 0.3409
Epoch 6/50
132/132 [==============================] - 0s 820us/step - loss: 1.4279 - acc: 0.4167
Epoch 7/50
132/132 [==============================] - 0s 813us/step - loss: 1.2360 - acc: 0.4924
Epoch 8/50
132/132 [==============================] - 0s 828us/step - loss: 1.2987 - acc: 0.4545
Epoch 9/50
132/132 [==============================] - 0s 845us/step - loss: 1.1682 - acc: 0.4924
Epoch 10/50
132/132 [==============================] - 0s 765us/step - loss: 1.1540 - acc: 0.5379
Epoch 11/50
132/132 [==========

In [45]:
pred = model.predict_classes(embedding_matrix_test)

In [46]:
float(sum(pred==Y_test))/embedding_matrix_test.shape[0]

0.5892857142857143

In [47]:
for ix in range(embedding_matrix_test.shape[0]):
    
    if pred[ix] != Y_test[ix]:
        print(ix)
        print test[0][ix],
        print emoji.emojize(emoji_dict[pred[ix]], use_aliases=True),
        print emoji.emojize(emoji_dict[Y_test[ix]], use_aliases=True)

5
['he', 'is', 'a', 'good', 'friend'] 😄 ❤
6
['I', 'am', 'upset'] 😞 ❤
7
['We', 'had', 'such', 'a', 'lovely', 'dinner', 'tonight'] 🍴 ❤
11
['work', 'is', 'hard'] ❤ 😞
12
['This', 'girl', 'is', 'messing', 'with', 'me'] 😄 😞
13
['are', 'you', 'serious', 'ha', 'ha'] ❤ 😄
21
['you', 'brighten', 'my', 'day'] 🍴 😄
23
['she', 'is', 'a', 'bully'] ❤ 😞
26
['I', 'worked', 'during', 'my', 'birthday'] 😄 😞
27
['My', 'grandmother', 'is', 'the', 'love', 'of', 'my', 'life'] 😄 ❤
28
['enjoy', 'your', 'break'] 😞 😄
29
['valentine', 'day', 'is', 'near'] 😄 ❤
30
['I', 'miss', 'you', 'so', 'much'] 😞 ❤
32
['My', 'life', 'is', 'so', 'boring'] 😄 😞
35
['he', 'can', 'pitch', 'really', 'well'] 😄 ⚾
37
['I', 'am', 'starving'] 😞 🍴
41
['I', 'like', 'your', 'jacket'] ❤ 😄
45
['I', 'love', 'to', 'the', 'stars', 'and', 'back'] ⚾ ❤
46
['What', 'you', 'did', 'was', 'awesome'] ⚾ 😄
48
['I', 'want', 'to', 'joke'] 😞 😄
50
['yesterday', 'we', 'lost', 'again'] ⚾ 😞
51
['family', 'is', 'all', 'I', 'have'] 😞 ❤
55
['I', 'did', 'not', 'have', '

In [55]:
x = ['i', 'do', 'think','this', 'class', 'is', 'very', 'interesting']

x_ = np.zeros((1,10,50))

for ix in range(len(x)):
    x_[0][ix] = embeddings_index[x[ix].lower()]

In [57]:
model.predict_classes(x_)

array([3])

In [76]:
model = Sequential()
model.add(LSTM(128, input_shape=(10,50), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(128, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_11 (LSTM)               (None, 10, 128)           91648     
_________________________________________________________________
dropout_13 (Dropout)         (None, 10, 128)           0         
_________________________________________________________________
lstm_12 (LSTM)               (None, 128)               131584    
_________________________________________________________________
dropout_14 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 5)                 645       
_________________________________________________________________
activation_6 (Activation)    (None, 5)                 0         
Total params: 223,877
Trainable params: 223,877
Non-trainable params: 0
_________________________________________________________________


In [77]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [78]:
hist = model.fit(embedding_matrix_train,Y_train,
                epochs = 50, batch_size=32,shuffle=True
                )

Epoch 1/50
132/132 [==============================] - 1s 8ms/step - loss: 1.5898 - acc: 0.2652
Epoch 2/50
132/132 [==============================] - 0s 3ms/step - loss: 1.5332 - acc: 0.3485
Epoch 3/50
132/132 [==============================] - 0s 2ms/step - loss: 1.5146 - acc: 0.3409
Epoch 4/50
132/132 [==============================] - 0s 2ms/step - loss: 1.4417 - acc: 0.4015
Epoch 5/50
132/132 [==============================] - 0s 2ms/step - loss: 1.3642 - acc: 0.4848
Epoch 6/50
132/132 [==============================] - 0s 2ms/step - loss: 1.2487 - acc: 0.5682
Epoch 7/50
132/132 [==============================] - 0s 2ms/step - loss: 1.0590 - acc: 0.6591
Epoch 8/50
132/132 [==============================] - 0s 3ms/step - loss: 0.8641 - acc: 0.7121
Epoch 9/50
132/132 [==============================] - 0s 2ms/step - loss: 0.7349 - acc: 0.7273
Epoch 10/50
132/132 [==============================] - 0s 2ms/step - loss: 0.6718 - acc: 0.7652
Epoch 11/50
132/132 [============================

In [79]:
pred = model.predict_classes(embedding_matrix_test)

In [80]:
float(sum(pred==Y_test))/embedding_matrix_test.shape[0]

0.6428571428571429

In [81]:
for ix in range(embedding_matrix_test.shape[0]):
    
    if pred[ix] != Y_test[ix]:
        print(ix)
        print test[0][ix],
        print emoji.emojize(emoji_dict[pred[ix]], use_aliases=True),
        print emoji.emojize(emoji_dict[Y_test[ix]], use_aliases=True)

3
['she', 'got', 'me', 'a', 'present'] 😄 ❤
5
['he', 'is', 'a', 'good', 'friend'] 😄 ❤
6
['I', 'am', 'upset'] 😞 ❤
7
['We', 'had', 'such', 'a', 'lovely', 'dinner', 'tonight'] 😄 ❤
11
['work', 'is', 'hard'] 😄 😞
12
['This', 'girl', 'is', 'messing', 'with', 'me'] ❤ 😞
13
['are', 'you', 'serious', 'ha', 'ha'] 😞 😄
16
['work', 'is', 'horrible'] 😄 😞
19
['any', 'suggestions', 'for', 'dinner'] 😄 🍴
21
['you', 'brighten', 'my', 'day'] ❤ 😄
23
['she', 'is', 'a', 'bully'] ❤ 😞
26
['I', 'worked', 'during', 'my', 'birthday'] 😄 😞
28
['enjoy', 'your', 'break'] ⚾ 😄
29
['valentine', 'day', 'is', 'near'] 😄 ❤
37
['I', 'am', 'starving'] 😞 🍴
40
['I', 'will', 'go', 'dance'] ⚾ 😄
41
['I', 'like', 'your', 'jacket'] ❤ 😄
48
['I', 'want', 'to', 'joke'] 😞 😄
49
['go', 'away'] ⚾ 😞
54
['You', 'totally', 'deserve', 'this', 'prize'] 😞 😄
